# 05-02. 교차 검증과 그리드 서치

## 검증 세트(validation set)
- 훈련 세트 중 일부를 검증 세트로 나누는 것
- 테스트 세트를 여러번 사용해서 평가할 경우 테스트 세트에만 잘 맞는 모델이 나오는 상황을 방지하기 위한 목적
- 훈련 세트로 모델을 훈련하고 검증 세트로 평가하고 테스트 세트는 최종 평가를 위해 한번만 사용

In [3]:
# 데이터셋 불러온 후 분할

import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2,random_state=42)

In [4]:
# 훈련 세트 중 일부를 검증 세트로 분할

sub_input, val_input, sub_target, val_target= train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


- 1040개의 샘플이 검증 세트로 분할

In [6]:
# 모델 훈련 및 평가

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


- 훈련 세트에 과대적합

## 교차 검증(corss validation)
- 검증 세트를 분할해 평가하는 과정을 여러번 반복하고 점수를 평균하여 최종 검증 점수를 얻는 방법
 - 분할된 각 세트를 한번씩 검증 세트로 사용한 이후 훈련 세트와 검증 세트를 다시 합친 전체 훈련 데이터로 모델을 훈련한 뒤 최종 테스트를 수행
- k-폴드 교차 검정(k-fold cross validation): 훈련 세트를 k 부분으로 나누어서 교차 검증을 수행하는 것   
보통 5-폴드 교차 검증이나 10-폴드 교차 검증을 많이 사용하며, k-겹 교차 검증이라고도 부름
- 사이킷런의 cross_validate(): 각각의 검증 동안 걸린 훈련 시간과 검증 시간, 점수를 반환하는 함수로 기본적으로 5-폴드 교차 검증을 수행함

In [9]:
# corss_validate() 함수 사용

from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01051378, 0.00801206, 0.00851107, 0.00826168, 0.00782442]), 'score_time': array([0.00135851, 0.00122023, 0.00129509, 0.0012114 , 0.00127459]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [10]:
# test_score 키에 담긴 5개의 점수를 평균하여 최종 점수를 계산

import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


- cross_validate()함수는 훈련 세트를 따로 섞지 않기 때문에 훈련 세트를 섞기 위해서는 분할기(splitter)를 지정해야 함   
이 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를, 분류 모델일 경우 StratifiedKFold를 사용

In [ ]:
# 기본적으로 cross_validate() 함수는 이런 형태로 실행

from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 StratifiedKFold의 매개변수를 조정해야함

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)  # n_split: 몇 폴드 교차 검증을 수행할 것인지/ shuffle: 섞을 것인지
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝
- 하이퍼파라미터 튜닝을 위해서는 매개변수를 조금씩 바꿔가며 훈려과 교차 검증을 수행해야 함
- 하나의 매개변수가 다른 매개변수에도 영향을 주기 때문에 최적의 값을 알아내려는 모든 매개변수를 동시에 바꿔가며 튜닝을 수행해야함


### DecisionTreeClassifier의 하이퍼파라미터
- max_depth: 트리의 최대 깊이(몇 개의 노드를 더 생성할 것인가)
- min_samples_split: 자식 노드를 생성하는데 필요한 최소 샘플의 수(default: 2)
- min_samples_leaf: 리프 노드가 되기 위한 최소 샘플 수(default: 1)
- min_weight_fraction_leaf: 리프 노드가 되기 위한 최소한의 샘플 비율(가중치가 부여된 샘플일 때 사용)
- max_features: 노드를 분할하기 위해 사용할 특성의 최대 개
- max_leaf_nodes: 리프 노드의 최대 개수
- min_impurity_decrease: 노드를 분할하기 위한 최소한의 불순도 감소량

### 그리드 서치(Grid Search)
- 지정된 하이퍼파라미터의 가능한 모든 조합을 시도하여 최적의 조합을 찾아내는 방법
- GridSearchCV: 사이킷런의 그리드 서치 클래스로 사용하면 하이퍼파라미터 탐색과 교차검증을 한 번에 수행할 수 있음
 - n_jobs를 통해 병렬 실행에 사용할 CPU 코어의 수 지정   
 값을 -1로 설정할 경우 시스템의 모든 CPU 코어 사용(default: 1)
- 탐색 과정
 - 1. 탐색할 매개변수 지정
 - 2. 훈련 세트에서 그리드 서치를 수행하여 최적의 매개변수 조합 탐색(그리드 서치 객체에 저장됨)
 - 3. 최상의 매개변수 조합으로 전체 훈련 세트를 사용해 최종 훈련 수행(모델은 그리드 서치 객체에 저장됨)

In [ ]:
# GridSearchCV 클래스로 매개변수 min_impurity_decrease의 최적값 탐색

from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0005, 0.0006]}  # 0.0001씩 증가하는 5개의 값 시도
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)  # 설정한 매개변수의 값을 바꿔가며 5-폴드 교차 검증 수행(5개의 값을 바꿔가며 5번씩 실행되므로 총 25번)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0005, 0.0006]})

In [ ]:
# GridSearchCV는 훈련이 끝나면 가장 점수가 높았던 모델에 전체 훈련 세트를 다시한번 훈련함

dt = gs.best_estimator_  # 가장 점수가 높았던 모델은 gs 객체의 best_estimator_에 저장
print(dt.score(train_input, train_target))
print(gs.best_params_)  # 최적의 매개변수는 best_params_에 저장
print(gs.cv_results_['mean_test_score'])  # 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_result_ 속성의 'mean_test_score' 키에 저장

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86761605 0.86338306]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


- best_parapms_와 동일한 매개변수가 출력

In [ ]:
# 더 복잡한 매개변수의 조합 탐색

params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),  # 원소 9개
          'max_depth': range(5, 20, 1),  # 원소 15개
          'min_samples_split': range(2, 100, 10)  # 원소 10개
          }  # 총 수행 교차 검증 회수: 5 x (9 x 15 x 5) = 6750

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)  # 최상의 매개변수 조합
print(np.max(gs.cv_results_['mean_test_score']))  # 최상의 교차 검증 점수

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


- 보완점: 탐색 범위를 설정한 특별한 근거가 없기 때문에 다른 범위나 간격으로 설정해야할 필요가 있음

### 랜덤 서치(Random Search)
- 매개변수값의 목록이 아닌 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달하는 방법
- 싸이파이의 randint(정수)와 uniform(실수) 클래스를 사용해 확률 분포 객체를 생성할 수 있음
- 그리드 서치보다 교차 검증의 수를 줄이면서 넓은 영역을 탐색 가능
- RandomizedSearchCV: 사이킷런의 랜덤 서치 클래스
 - n_iter: 확률 분포 범위 내에서 수행할 샘플링 횟수

In [2]:
# randint 클래스 사용

from scipy.stats import uniform, randint

rgen = randint(0, 10)  # 0부터 10 사이의 범위
rgen.rvs(10)  # 10번 샘플링

array([4, 9, 1, 9, 2, 6, 4, 2, 6, 1])

In [ ]:
# 샘플링 횟수를 늘리면 고르게 샘플링 됨을 확인할 수 있음
# 분포를 고려하지 않는 numpy.random.randint와의 차이점
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 92,  96,  99, 109,  95, 120, 101,  92,  94, 102]))

In [ ]:
# uniform 클래스 사용

ugen = uniform(0, 1)  # 0부터 1 사이의 범위
ugen.rvs(10)  # 10번 샘플링

array([0.81999903, 0.83599486, 0.58361263, 0.33102348, 0.47564815,
       0.02850646, 0.00765577, 0.67213617, 0.65026361, 0.97696721])

In [11]:
# RandomizedSearchCV 클래스로 최적의 매개변수 조합 탐색
from sklearn.model_selection import RandomizedSearchCV

params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)
          }

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)  # 최적의 매개변수 조합
print(np.max(gs.cv_results_['mean_test_score']))  # 최상의 교차 검증 점수

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [ ]:
# 최종 모델의 테스트 성능 확인

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


- 테스트 세트 점수는 일반적으로 검증 세트에 대한 점수보다 조금 작다

In [12]:
# DecisionTreeClassifier의 splitter 매개변수를 random으로 바꾸면 노드를 무작위로 분할한 다음 가장 좋은 것을 선택(default는 'best'로 최선의 분할을 탐색)

gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)  # 최적의 매개변수 조합
print(np.max(gs.cv_results_['mean_test_score']))  # 최상의 교차 검증 점수

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981


- 교차 검증 점수가 splitter='best'에 비해 낮아짐